# Préparation des variables de mode de vie

In [1]:
library(tidyverse)
library(factoextra)

# Charger les données
DataPilot <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/01_pilot_2025.rds")

# Charger le fichier des variables
source("01-1_variables.R")

df_mode_de_vie <- DataPilot %>%
  select(all_of(variables_mode_de_vie))

-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.1
v ggplot2   3.5.1     v tibble    3.2.1
v lubridate 1.9.3     v tidyr     1.3.1
v purrr     1.0.2     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa



In [2]:
library(dplyr)

df_mode_de_vie %>%
  select(starts_with("lifestyle_typeTransport")) %>%
  summarise(across(everything(), \(x) sum(x, na.rm = TRUE)))
#  summarise(across(everything(), sum, na.rm = TRUE))

lifestyle_typeTransportCar,lifestyle_typeTransportSUV,lifestyle_typeTransportMoto,lifestyle_typeTransportWalk,lifestyle_typeTransportBicycle,lifestyle_typeTransportPublicTransit
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
912,298,8,131,19,184


In [3]:
# Boucle pour afficher la table pour chaque variable
for (v in variables_mode_de_vie) {
  if (v != "id") {
    cat("Table pour la variable:", v, "\n")
    print(table(df_mode_de_vie[[v]]))
    cat("\n\n")
  }
}

Table pour la variable: ses_dwellingApp 

   0    1 
1137  180 


Table pour la variable: ses_dwellingLoft 

   0    1 
1306   11 


Table pour la variable: ses_dwellingCondo 

   0    1 
1191  126 


Table pour la variable: ses_dwellingTour 

   0    1 
1252   65 


Table pour la variable: ses_dwellingDetachedHouse 

  0   1 
647 670 


Table pour la variable: ses_dwellingTownhouse 

   0    1 
1201  116 


Table pour la variable: ses_dwellingDuplex 

   0    1 
1250   67 


Table pour la variable: ses_dwellingCoop 

   0    1 
1307   10 


Table pour la variable: ses_dwellingHLM 

   0    1 
1308    9 


Table pour la variable: ses_dwellingMobile 

   0    1 
1301   16 


Table pour la variable: ses_dwellingOther 

   0    1 
1270   47 


Table pour la variable: lifestyle_typeTransportCar 

  0   1 
652 912 


Table pour la variable: lifestyle_typeTransportSUV 

   0    1 
1266  298 


Table pour la variable: lifestyle_typeTransportMoto 

   0    1 
1556    8 


Table pour la variabl

### Transformations

- Regroupement de ses_dwelling
  - ses_dwelling_Condo : ses_dwelling_Condo + ses_dwelling_Loft + ses_dwelling_Tour
  - ses_dwelling_Other : ses_dwelling_Other + ses_dwelling_Coop + ses_dwelling_HLM + ses_dwelling_Mobile

- Regroupement des act_transport
  - act_transport_Car : act_transport_Car + act_transport_Moto (parce qu'il y avraiment trop peu de moto, assigner la classe la plus populeuse)
  - Les autres ne sont pas regroupés

In [4]:
# Regroupement de ses_dwelling
#-----------------------------

df_mode_de_vie <- df_mode_de_vie %>%

  mutate(
    ses_dwellingCondo = if_else(
      ses_dwellingLoft == 1 | ses_dwellingTour == 1,
      1,
      ses_dwellingCondo
    ),
    ses_dwellingOther = if_else(
      ses_dwellingCoop == 1 | ses_dwellingHLM == 1 | ses_dwellingMobile == 1,
      1,
      ses_dwellingOther
    )
  ) %>%
  select(
    -ses_dwellingLoft,
    -ses_dwellingTour,
    -ses_dwellingCoop,
    -ses_dwellingHLM,
    -ses_dwellingMobile
  )

In [5]:
# Regroupement de act_transport
#------------------------------

df_mode_de_vie <- df_mode_de_vie %>%

  mutate(
    lifestyle_typeTransportCar = if_else(
      lifestyle_typeTransportMoto == 1,
      1,
      lifestyle_typeTransportCar
    )) %>%
  select(
    -lifestyle_typeTransportMoto
  )

In [6]:
variables_mode_de_vie_clust <- c(
  "id",
  #"postal_code", # À transformer en rural, urbain, région, banlieue
  "ses_dwellingApp",
  "ses_dwellingCondo",
  "ses_dwellingDetachedHouse",
  "ses_dwellingTownhouse",
  "ses_dwellingDuplex",
  "ses_dwellingOther",
  "lifestyle_typeTransportCar",
  "lifestyle_typeTransportSUV",
  "lifestyle_typeTransportWalk",
  "lifestyle_typeTransportBicycle",
  "lifestyle_typeTransportPublicTransit"
)

In [7]:
# Boucle pour afficher la table pour chaque variable
for (v in variables_mode_de_vie_clust) {
  if (v != "id") {
    cat("Table pour la variable:", v, "\n")
    print(table(df_mode_de_vie[[v]]))
    cat("\n\n")
  }
}

Table pour la variable: ses_dwellingApp 

   0    1 
1137  180 


Table pour la variable: ses_dwellingCondo 

   0    1 
1115  202 


Table pour la variable: ses_dwellingDetachedHouse 

  0   1 
647 670 


Table pour la variable: ses_dwellingTownhouse 

   0    1 
1201  116 


Table pour la variable: ses_dwellingDuplex 

   0    1 
1250   67 


Table pour la variable: ses_dwellingOther 

   0    1 
1235   82 


Table pour la variable: lifestyle_typeTransportCar 

  0   1 
644 920 


Table pour la variable: lifestyle_typeTransportSUV 

   0    1 
1266  298 


Table pour la variable: lifestyle_typeTransportWalk 

   0    1 
1433  131 


Table pour la variable: lifestyle_typeTransportBicycle 

   0    1 
1545   19 


Table pour la variable: lifestyle_typeTransportPublicTransit 

   0    1 
1380  184 




In [8]:
# Sauvegarder les données préparées
saveRDS(df_mode_de_vie, file = "../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/02_pilot_2025_mode_de_vie.rds")

---

# SUGGESTION : Activité de Clustering

Si souhaité, effectuer un exercice de clustering avec ce sous-groupe de variables pour mieux saisir les données et leur relation. 

Cet exercice est purement exploratoire et sert à voir si les données peuvent être utilisées pour faire du clustering.